In [ ]:
# 국세청_사업자등록정보 진위확인 및 상태조회 서비스
# https://www.data.go.kr/data/15081808/openapi.do#tab_layer_detail_function

# 참고사이트 : Convert curl commands to Python, JavaScript and more
# https://curlconverter.com/

In [ ]:
# API에 빈번한 호출이 일어날 경우 API서버가 (사람이 아닌)로봇이 호출하는 것으로 인식하고 connection을 차단함.
# 이를 해결하기 위해 fake_useragent 패키지를 설치함
# pip install fake_useragent==1.5.1

In [1]:
# 필요한 라이브러리 Import
import pandas as pd
import requests
from datetime import date
from fake_useragent import UserAgent
from IPython.display import clear_output

# csv파일을 String(object)으로 읽음
dfRaw = pd.read_csv("BRNO_CORP_BIZ_DATA_20240423.csv", dtype = object)

# 주민번호로 판단되는 사업자번호 제거
juminRulePattern = '\d{2}([0]\d|[1][0-2])([0][1-9]|[1-2]\d|[3][0-1])[1-8]\d{6}'  # 주민번호 판단을 위한 정규식
# \d{2} : 맨앞 정수 2자리(생년)은 어떤 정수값이 와도 상관없음
# ([0]\d|[1][0-2]) : 첫자리가 0인 경우는 뒤에 어떤 정수가 와도 괜찮음. 첫자리가 1인 경우 뒷자리는 0, 1, 2만 올 수 있음 (01-12 생월을 표현)
# ([0][1-9]|[1-2]\d|[3][0-1]) : 생일은 첫자리가 0이면 뒷자리가 0이 될 경우 0일이 되기 때문에 0 다음에는 1-9만 올 수 있음
# [1-8] : 주민번호 뒷자리 첫번째 숫자는 1~8만 갖는다 (내국인: 1~4, 외국인: 5~8)
# \d{6} : 주민번호 첫자리를 제외한 숫자는 총 6자리다
dfRaw['isJuminNo'] = dfRaw.BRNO.str.match(pat=f'({juminRulePattern})')  # 주민번호 판단을 위한 정규식 적용
df = dfRaw.loc[dfRaw["isJuminNo"] == False]  # 주민번호가 아니라고 판단되는 것들만 추출

filteredDf = pd.DataFrame()
filteredDf['b_no'] = df['BRNO']
print(f'전체 검사 대상 사업자번호 수는 {len(filteredDf)} 입니다')


# 사업자번호가 0으로 시작되는 것은 사업자번호 부여규칙에 위배되므로 이는 호출 대상에서 제외
# 사업자등록번호 10자리(xxx-xx-xxxxx) 부여규칙
# 일련번호코드(3자리)
# 신규사업자에게 사용 가능한 번호 101~999를 순차적으로 부여

# 개인/법인 구분코드(2자리)
# - 개인구분코드
# (1) 개인과세사업자는 특정 동 구별 없이 01~79까지를 순차적으로 부여
# (2) 개인면세사업자는 산업 구분없이 90~99까지를 순차적으로 부여
# (3) 소득세법 제2조 제3항에 해당하는 법인이 아닌 종교 단체 : 89
# (4) 소득세법 제2조 제3앙에 핻아하는 자로서 (3)이외의자(아파트관리사무소 등) 및 다단계판매원 : 80
# - 법인성격코드
# (1) 영리법인의 본점 : 81, 86, 87, 88
# (2) 비영리법인의 본점 및 지점(법인격 없는 사단, 재단, 기타 단체 중 법인으로 보는 단체를 포함) : 82
# (3) 국가, 지방자치단체, 지방자치단체조합 : 83
# (4) 외국법인의 본/지점 및 연락사무소 : 84
# (5) 영리법인의 지점 : 85

# 일련번호코드(4자리)
# - 과세사업자(일반과세자, 간이과세자), 면세사업자, 법인사업자별로 등록 또는 지정 일자순으로 사용가능한 번호를 0001~9999로 부여

# 검증번호(1자리)
# 전산시스템에 의하여 사업자등록번호의 오류여부를 검증하기 위하여 1자리의 검증번호를 부여
filteredDf = filteredDf[filteredDf.b_no.str[0] != '0']
filteredDf = filteredDf.reset_index(drop=True)
print(f'사업자번호가 0으로 시작하는 기업을 제외한 검사 대상 사업자번호 수는 {len(filteredDf)} 입니다')

전체 검사 대상 사업자번호 수는 7699673 입니다
사업자번호가 0으로 시작하는 기업을 제외한 검사 대상 사업자번호 수는 7694164 입니다


In [ ]:
today = date.today()

# 서비스키를 parameter로 전달하기 위함
params = {
    'serviceKey': '6No4JLBZPafGmvs0YQ/WiHSNqVrvqqAox0frrHQJbt/vc6QT4xzf2Idnc9CaktyAdlI3FYGILmdhVIH9u5ck2Q=='
}
# 헤더정보
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
}
  
# URL요청 함수
# API서버가 request를 로봇으로 인식하면 connection을 차단하므로, 이를 예외처리함(User-Agent 정보를 변경하여 재귀호출)
def urlRequest(headersRx):
    try:
        response = requests.post('https://api.odcloud.kr/api/nts-businessman/v1/status', params=params, headers=headersRx, json=json_data)
        return response
    except:
        headers = {
            "accept": "application/json",
            "Content-Type": "application/json",
            "User-Agent": UserAgent().random
        }
        return urlRequest(headers)


# 사업자번호를 100개 단위로 잘라서 처리(국세청 API에서 100개 단위로 처리하는 것이 MAX임)
dfReturnValue = pd.DataFrame()
for n in range(0, len(filteredDf), 100):
    clear_output(wait=True) # output cell을 clear함
    print(f'진행중: {n} 번째')
    endPoint = min(n + 100, len(filteredDf) + 1)
    dfBRNO = filteredDf.iloc[n:endPoint, :] # 사업자번호를 100개 단위로 자름
    json_data = dfBRNO.to_dict('list') # 잘라낸 사업자번호 100개를 dictionary 형태로 변경
    response = urlRequest(headers) # headers값을 parameter로 urlRequest함수를 호출하고 휴폐업조회 결과값을 return받음  
    jsResponse = response.json()['data']
    dfReturnValue = pd.concat([dfReturnValue, pd.DataFrame(jsResponse)], axis=0, ignore_index = True)
    display(dfReturnValue) # 현재 진행상태를 파악하기 위한 출력
dfApiCallResult = dfReturnValue[['b_no', 'b_stt_cd']] # b_stt_cd (1:정상, 2:휴업, 3:폐업)

# 크롤링 날짜를 기준으로 기준년월(GG_YM)값을 삽입하고 그 결과를 csv로 내보냄
ggYm = today.strftime("%Y") + today.strftime("%m")
dfApiCallResult.insert(0, column='GG_YM', value=ggYm)
dfApiCallResult.to_csv(f"IT_EFAS_BIZ_STT_{today}.csv", index=False) # 인덱스를 지우고 csv로 저장

진행중: 51700 번째
